<a href="https://colab.research.google.com/github/Fizza-Rubab/CS432-GPU/blob/main/HWs/assignment1_fr06161.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
## Fizza Rubab

### Question 1
The first task asks you to write CUDA code to initialize a random array of 1000000 elements.
Provide the following:

#### Part (1)
Share the amount of time it takes to populate data in the array on the CPU. Use any
timing function like clock() or hiresolution counters to note the amount of time it takes to
generate data on the CPU. (+20)


In [8]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4pz2ehsx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4pz2ehsx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=5cf92bf5e27dfa0ac341d277e4cb009f4f9e94df7b76cb1b928c29c84bf39172
  Stored in directory: /tmp/pip-ephem-wheel-cache-jco9ylfe/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [9]:
%%cu
#include <bits/stdc++.h>
#include <stdio.h>

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
 

__global__ void MyKernel(int* data) 
{
    
}

int main()
{
  const int N = 1000000;
  clock_t start, end;
  start = clock();
  int array[N];
  for (int i=0; i<N; i++)
    array[i] = rand();
  end = clock();
  double time_elapsed = double(end - start) / double(CLOCKS_PER_SEC);
  printf("Time taken to generate a 1000000 element random array is %f seconds.", time_elapsed);
  return 0;
}


Time taken to generate a 1000000 element random array is 0.016548 seconds.


#### References
1. https://www.geeksforgeeks.org/measure-execution-time-with-high-precision-in-c-c/
2. https://www.tutorialspoint.com/c_standard_library/c_function_rand.htm

### Part(2)
Find a way to do the random data initialization of the array on the GPU. Do not just
copy data from CPU array to GPU. Instead, you should use any library like cuRand but
you should provide relevant references. Note the time it takes to run the kernel using the
CUDA event API to calculate the amount of time needed by the GPU. (+20)

In [18]:
%%writefile /content/test.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand_kernel.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void device_api_kernel(curandState *states, float *out, int N)
{
    int i;
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int nthreads = gridDim.x * blockDim.x;
    curandState *state = states + tid;

    curand_init(9384, tid, 0, state);

    for (i = tid; i < N; i += nthreads)
    {
        float rand = curand_uniform(state);
        rand = rand * 2;
        out[i] = rand;
    }
}


int main(int argc, char **argv)
{
    int N = 1000000;
    curandState *states = NULL;
    float *dOut, *hOut;
	  int threads_per_block =  128;
	  int blocks_per_grid = ceilf(N/threads_per_block + 1);
    cudaMalloc((void **)&dOut, sizeof(float) * N);
    cudaMalloc((void **)&states, sizeof(curandState)*threads_per_block * blocks_per_grid);
    hOut = (float *)malloc(sizeof(float) * N);
    device_api_kernel<<<blocks_per_grid, threads_per_block>>>(states, dOut, N);
    checkCudaErr(cudaMemcpy(hOut, dOut, sizeof(float) * N, cudaMemcpyDeviceToHost), "memcopy");
    printf("Displaying first 10 random numbers from array of 1000000 elements\n");
    for (int i = 0; i < 10; i++)
        printf("%2.4f\n", hOut[i]);
    free(hOut);
    cudaFree(dOut);
    cudaFree(states);
    return 0;
}

Overwriting /content/test.cu


In [19]:
!nvcc /content/test.cu -lcurand -o test

In [20]:
!./test

Displaying first 10 random numbers from array of 1000000 elements
1.7875
1.0698
1.5584
0.7782
0.3320
0.5016
1.3649
0.3079
0.1805
0.2031


### Part(3)
Copy the random array data generated on the GPU from the GPU to the CPU using
cudaMemcpy. Note the amount of time it takes to move the data from GPU to the CPU.
(+10)

In [17]:
%%writefile /content/test.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand_kernel.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void device_api_kernel(curandState *states, float *out, int N)
{
    int i;
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int nthreads = gridDim.x * blockDim.x;
    curandState *state = states + tid;

    curand_init(9384, tid, 0, state);

    for (i = tid; i < N; i += nthreads)
    {
        float rand = curand_uniform(state);
        rand = rand * 2;
        out[i] = rand;
    }
}


int main(int argc, char **argv)
{
    int N = 1000000;
    curandState *states = NULL;
    float *dOut, *hOut;
	  int threads_per_block =  128;
	  int blocks_per_grid = ceilf(N/threads_per_block + 1);
    cudaMalloc((void **)&dOut, sizeof(float) * N);
    cudaMalloc((void **)&states, sizeof(curandState)*threads_per_block * blocks_per_grid);
    hOut = (float *)malloc(sizeof(float) * N);
    device_api_kernel<<<blocks_per_grid, threads_per_block>>>(states, dOut, N);
    checkCudaErr(cudaMemcpy(hOut, dOut, sizeof(float) * N, cudaMemcpyDeviceToHost), "memcopy");
    printf("Displaying first 10 random numbers from array of 1000000 elements\n");
    for (int i = 0; i < 10; i++)
        printf("%2.4f\n", hOut[i]);
    free(hOut);
    cudaFree(dOut);
    cudaFree(states);
    return 0;
}

Overwriting /content/test.cu


## Question 2
Write CUDA code to calculate the sum of 1000 elements array and output the sum. Calculate
the total time for doing the calculation on the CPU as well as on the GPU. Use an appropriate
execution configuration when you launch your kernel. Share the following information from your
solution.

1. Code compiles and outputs as expected both on CPU and GPU. (+10)
2. Plot your results in a graph and show both your results (CPU/GPU) against different
dataset sizes (1000,10000,100000) and different execution configurations (+30)
3. Do proper error handling in your code so any CUDA error should be reported to the user.
(+10)

